In [ ]:
import phasr as phr

In [ ]:
phr.__version__

In [ ]:
import numpy as np
pi=np.pi
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from IPython.display import display, Math

In [ ]:
def print_radius(charge_dict,manual_digits_offset=0):
    redchisq=charge_dict['redchisq']
    
    r_ch=charge_dict['r_ch']
    
    dr=charge_dict['dr_ch_stat']*np.sqrt(redchisq)
    dr_s=charge_dict['dr_ch_syst']
    dr_m=charge_dict['dr_ch_model']
    dr_u=charge_dict['dr_ch_syst_upper']
    dr_l=charge_dict['dr_ch_syst_lower']
    dr_d=charge_dict['dr_ch_dist']
    dr_d_u=charge_dict['dr_ch_dist_upper']
    dr_d_l=charge_dict['dr_ch_dist_lower']
    dr_dm=np.sqrt(dr**2+dr_d**2)
    
    digs_r=int(-np.log10(dr))+2
    
    while np.max([dr,dr_s,dr_m])*10**digs_r>69: 
        digs_r-=1
    while np.max([dr,dr_s,dr_m])*10**digs_r<3.1:
        digs_r+=1

    digs_r+=manual_digits_offset
    
    s=r'\sqrt{\langle r^2 \rangle} = '+r'{r_ch:1.{digs_r}f}({dr:1d})'.format(r_ch=r_ch,digs_r=digs_r,dr=int(np.round(dr*10**digs_r,0)))+r'\begin{matrix}'+r'({dr_s:1d}) [{dr_m:1d}] \\ ( +{dr_d_u:1d}/-{dr_d_l:1d} ) [{dr_dm:1d}]'.format(dr_s=int(np.round(dr_s*10**digs_r,0)),dr_m=int(np.round(dr_m*10**digs_r,0)),dr_d_u=int(np.round(dr_d_u*10**digs_r,0)),dr_d_l=int(np.round(dr_d_l*10**digs_r,0)),dr_dm=int(np.round(dr_dm*10**digs_r,0)))+r'\end{matrix}'
    display(Math(s))

# Import data

## Cross section Data

In [ ]:
Z=54
A=132
datasets={'Scrit_2018':{'path':"./Scrit_Xe132_2018.txt",'fit_luminosities':'y','luminosities':{151.:0.87*1e1, 201.:1.06*1e1, 301.:1.55*1e1},
                        'luminosities_bounds':{151.:(0.2*1e1,10.*1e1),201.:(0.2*1e1,12.5*1e1),301.:(0.2*1e1,12.0*1e1)}}}

In [ ]:
#How to anser the questions: 
#What column (starting at 0) contains the central values for the energie? 0
#In what units is the energy (MeV or GeV)? MeV
#What column (starting at 0) contains the central values for the angles? 1
#In what units is the angle theta (deg or rad)? rad
#Do the cross sections need to be fitted with a free luminosity parameter for each energy? (y or n) y
#What column (starting at 0) contains the energies for the cross section*luminosity points? 2
#In what units is the cross section*luminosity (1/s,1/ms,1/micros)? 1/s
#Does the data distinguish between statistical and systematical uncertainties? (y or n) y
#What columns (starting at 0), if any, contain statistical uncertainties for the cross sections (separate by comma)? 3
#Are the statistical uncertainties absolute values or relative to the central value? (answer with: absolute or relative) relative
#Are the relative statistical uncertainties in percent and thus need to divided by 100? (y or n) y
#What columns (starting at 0), if any, contain systematical uncertainties for the cross sections (separate by comma)? skip
#Are the systematical uncertainties absolute values or relative to the central value? (answer with: absolute or relative) skip
#Are the relative systematical uncertainties in percent and thus need to divided by 100? (y or n) y
#Do you want to add a global relative statistical uncertainty w.r.t. the cross section as an (additional) uncertainty component? (For 3% insert 3, type 0 if you do not want to do so)? 0
#Do you want to add a global relative systematical uncertainty w.r.t. the cross section as an (additional) uncertainty component? (For 3% insert 3, type 0 if you do not want to do so)? 5

In [ ]:
dataset=datasets[list(datasets.keys())[0]]
#phr.import_dataset(dataset['path'],list(datasets.keys())[0],Z,A,fit_luminosity=dataset['fit_luminosity'])

### Compare Data to reference parameterization

In [ ]:
# Comparison extraction from dVries et al. 1987
#nucleus = phr.nuclei.load_reference_nucleus(Z,A)
#nucleus
nucleus=phr.nucleus('Xe132_FB',Z=Z,A=A,ai=np.array([ 0.05740165,  0.05250304, -0.03442148, -0.02282753,  0.01333736]),R=10)

In [ ]:
datasets.keys()

In [ ]:
data_name=list(datasets.keys())[0]
dataset=datasets[data_name]
data, corr_stat, corr_syst = phr.cross_section_fitter.data_prepper.load_dataset(data_name,Z,A)

In [ ]:
args_optimised = {'method': np.str_('DOP853'), 'N_partial_waves': np.int64(70), 'atol': np.float64(1e-06), 'rtol': np.float64(1e-06), 'energy_norm': np.float64(0.01973269804), 'phase_difference_limit': np.float64(1e-06)}

In [ ]:
theta=np.arange(25,120,1)*pi/180
CS={}
L={}
L=datasets[data_name]['luminosities']
for E in np.unique(data[:,0]):
    CS[E]=L[E]*phr.crosssection_lepton_nucleus_scattering(E,theta,nucleus,**args_optimised)*phr.constants.hc**2

In [ ]:
for E in np.unique(data[:,0]):
    mask = data[:,0]==E
    plt.errorbar(data[mask,1]*180/pi,data[mask,2],np.sqrt(data[mask,3]**2+data[mask,4]**2),capsize=2,marker='_',linestyle='',label='E='+str(E)+'MeV')
first = True
for E in np.unique(data[:,0]):
    plt.plot(theta*180/pi,CS[E],color='gray',zorder=-2,label='dVries' if first else None)
    first=False
plt.yscale('log')
plt.legend()
plt.ylim(1e-6,1e1)
plt.title(r'$^{40}$Ar')
plt.xlabel(r'$\theta$ in deg')
plt.ylabel(r'cross section in fm$^{2}$/sr')

In [ ]:
t=2.71
c=5.42
rho_0=0.07179364630887028
def rho_Xe132(r):
    return rho_0/(1+np.exp(4*np.log(3)*(r-c)/t))

nucleus_num = phr.nucleus('132Xe_num',Z=54,A=132,charge_density=rho_Xe132)
nucleus_num.fill_gaps()
nucleus_num.set_electric_field_from_charge_density()
nucleus_num.set_electric_potential_from_electric_field()
nucleus_num.set_Vmin()

In [ ]:
energy=301.0
theta=np.arange(30,60,1e-1)
plt.plot(theta,phr.crosssection_lepton_nucleus_scattering(energy,theta*pi/180,nucleus_num)*phr.constants.hc**2,label=r'$E=249.5~$MeV')
for E in np.unique(data[:,0]):
    mask = data[:,0]==energy
    if E==energy:
        plt.errorbar(data[mask,1]*180/pi,data[mask,2]/L[E],np.sqrt(data[mask,3]**2+data[mask,4]**2),capsize=2,marker='_',linestyle='',label='E='+str(E)+'MeV')
plt.yscale('log')
plt.title(r"Cross section $^{48}$Ti")
plt.xlabel(r"$\theta$ in deg")
plt.ylabel(r"$\frac{\operatorname{d}<w\sigma}{\operatorname{d}\Omega}$ in MeV$^{-2}$")
plt.legend()
plt.show()

## Barrett Moment

In [ ]:
path_example = "./Fricke1995_barrett_moments.txt"

with open( path_example, "rb" ) as file:
    barrett_moments_input = np.genfromtxt( file,dtype=[r'<U3',int,int,r'<U10']+9*[float],names=['Symbol','Z','A','transition','E_tran','dE_tran','N_pol','alpha','k','C_z','barrett','dbarrett_stat','dbarrett_pol'])

pd.DataFrame(barrett_moments_input)

In [ ]:
barrett_moment_Ar40 = barrett_moments_input[np.logical_and(barrett_moments_input['Z']==18,barrett_moments_input['A']==40)]
barrett_moment_Ar40_dict = {key:barrett_moment_Ar40[key][0] for key in  ['Z','A','k','alpha','barrett']}
barrett_moment_Ar40_dict['dbarrett'] = np.sqrt(barrett_moment_Ar40['dbarrett_stat'][0]**2 + barrett_moment_Ar40['dbarrett_pol'][0]**2)

In [ ]:
barrett_moment_Ar40_dict

In [ ]:
phr.cross_section_fitter.data_prepper.import_barrett_moment("Fricke1995_2p",**barrett_moment_Ar40_dict)

# Base fits

In [ ]:
# Check available datasets

In [ ]:
phr.cross_section_fitter.data_prepper.list_datasets(Z,A)

In [ ]:
phr.cross_section_fitter.data_prepper.list_barrett_moments(Z,A)

In [ ]:
args_optimised =  {'method': 'DOP853',
 'N_partial_waves': 25,
 'atol': 1e-08,
 'rtol': 1e-06,
 'energy_norm': 0.001973269804,
 'phase_difference_limit': 1e-06}

In [ ]:
results_dict = phr.cross_section_fitter.fit_organizer.parallel_fitting_automatic(datasets,Z,A,np.arange(9.00,9.75,0.25),redo_N=True,redo_aggressive=True,N_base_offset=1,N_base_span=0,cross_section_args=args_optimised,ai_ini=nucleus.ai)

Starting current fit step (R=9.0,N=6) with loss = 23175.80545258069
Starting current fit step (R=9.25,N=6) with loss = 6012.191707709386
Starting current fit step (R=9.5,N=6) with loss = 2030.3411761697344
Loss (R=9.0,N=6,eval:10) = 23175.80548084752
Loss (R=9.25,N=6,eval:10) = 6012.191713822123
Loss (R=9.5,N=6,eval:10) = 2030.3411772351171
Loss (R=9.0,N=6,eval:20) = 16668.128769703402
Loss (R=9.25,N=6,eval:20) = 6004.327484839674
Loss (R=9.5,N=6,eval:20) = 2157.268800638729
Loss (R=9.0,N=6,eval:30) = 9216.828972526528
Loss (R=9.25,N=6,eval:30) = 5996.8473185120865
Loss (R=9.5,N=6,eval:30) = 2030.3037478855745
Loss (R=9.0,N=6,eval:40) = 9214.301235743396
Loss (R=9.25,N=6,eval:40) = 5859.9213958011405
Loss (R=9.5,N=6,eval:40) = 2030.267023843519
Loss (R=9.0,N=6,eval:50) = 15084.168706083705
Loss (R=9.25,N=6,eval:50) = 5858.1094005544655
Loss (R=9.5,N=6,eval:50) = 2084.578320129608
Loss (R=9.0,N=6,eval:60) = 9213.496143991062
Loss (R=9.25,N=6,eval:60) = 7717.556699651228
Loss (R=9.5,N=6,

In [ ]:
database_tables, Rs, Ns, hist = phr.cross_section_fitter.generate_data_tables(results_dict)

In [ ]:
pd.DataFrame(database_tables['p_val'],index=Rs,columns=Ns)

In [ ]:
plt.title('fit distribution')
plt.xlabel(r'$p_{val}$')
plt.plot([0.2,0.2],[0,42],color='red',linestyle='--')
plt.ylim(0,42)
plt.xlim(0,0.7)
plt.hist(hist['p_val'],10)
plt.show()

In [ ]:
selected_RN_tuples = phr.cross_section_fitter.fit_organizer.select_RN_based_on_property(results_dict,'p_val',0.2)

In [ ]:
len(selected_RN_tuples)

# Final fits (with barrett moments)

In [ ]:
ini_settings = {'datasets_barrett_moment':[],'monotonous_decrease_precision':np.inf}

In [ ]:
results_dict_bar = phr.cross_section_fitter.fit_organizer.parallel_fitting_manual(datasets,Z,A,selected_RN_tuples,redo_N=True,monotonous_decrease_precision=0.04,barrett_moment_keys=['Fricke1995_2p'],initialize_from=ini_settings,cross_section_args=args_optimised)

In [ ]:
database_tables_bar, Rs, Ns, hist_bar = phr.cross_section_fitter.generate_data_tables(results_dict_bar)

In [ ]:
pd.DataFrame(database_tables_bar['p_val'],index=Rs,columns=Ns)

In [ ]:
plt.title('fit distribution')
plt.xlabel(r'$p_{val}$')
plt.ylim(0,11)
plt.xlim(0,0.8)
plt.plot([0.3,0.3],[0,42],color='red',linestyle='--')
plt.hist(hist_bar['p_val'],20)
plt.show()

In [ ]:
qs=[50,235,370,600]
results_dict_bar_sel, results_dict_bar_veto, veto_fct = phr.cross_section_fitter.fit_organizer.split_based_on_asymptotic_and_p_val(results_dict_bar,qs=qs,p_val_lim=0.3)

In [ ]:
# check asymptotics choice

In [ ]:
q = np.arange(1e-1,1000,1e-1)
for key in results_dict_bar_sel:
    result=results_dict_bar_sel[key]
    nuc_result = phr.nucleus('Ar40_FB_test_'+key,18,40,ai=result['ai'],R=result['R'])
    plt.plot(q,np.abs(nuc_result.form_factor(q)),color='C0',zorder=1)
for key in results_dict_bar_veto:
    result=results_dict_bar_veto[key]
    nuc_result = phr.nucleus('Ar40_FB_test_'+key,18,40,ai=result['ai'],R=result['R'])
    plt.plot(q,np.abs(nuc_result.form_factor(q)),color='gray',zorder=-3)
for qi in qs:
    plt.plot([qi,qi],[1e-8,1e0],color='red',zorder=-5,linestyle='--')
plt.plot(q,veto_fct(q),color='C1')
plt.xlim(0,1000)
plt.ylim(1e-8,1e0)
plt.xlabel('form factor')
plt.xlabel('$q$ in MeV')
plt.yscale('log')

In [ ]:
# select best result
best_key_bar = None
best_p_val = 0
for key in results_dict_bar_sel:
    p_val = results_dict_bar_sel[key]['p_val']
    if p_val > best_p_val:
        best_p_val = p_val
        best_key_bar = key
best_key_bar, best_p_val

In [ ]:
phr.cross_section_fitter.pickler.promote_best_fit(best_key_bar,results_dict_bar_sel)

In [ ]:
best_results_dict_bar = phr.cross_section_fitter.uncertainties.add_systematic_uncertainties(best_key_bar,results_dict_bar_sel)

In [ ]:
best_nuc_result = phr.nucleus('Ar40_FB_best_test',18,40,ai=best_results_dict_bar['ai'],R=best_results_dict_bar['R'])
best_cov_ai_model = best_results_dict_bar['cov_ai_model']
best_cov_ai_stat = best_results_dict_bar['cov_ai_stat'] * np.sqrt(best_results_dict_bar['redchisq'])
best_cov_ai_syst = best_results_dict_bar['cov_ai_syst']

In [ ]:
r = np.arange(0,15,1e-2)
best_rho = best_nuc_result.charge_density(r)
plt.plot(r,best_rho,color='C0',zorder=2,label='best fit')
best_jac_rho = best_nuc_result.charge_density_jacobian(r)
best_drho_model = np.sqrt(np.einsum('ij,ik,kj->j',best_jac_rho,best_cov_ai_model,best_jac_rho))
best_drho_stat = np.sqrt(np.einsum('ij,ik,kj->j',best_jac_rho,best_cov_ai_stat,best_jac_rho))
best_drho_syst = np.sqrt(np.einsum('ij,ik,kj->j',best_jac_rho,best_cov_ai_syst,best_jac_rho))
plt.plot(r,best_rho+best_drho_stat,color='C0',linestyle=':',zorder=1,label='stat. error')
plt.plot(r,best_rho-best_drho_stat,color='C0',linestyle=':',zorder=1)
plt.plot(r,best_rho+best_drho_syst,color='C0',linestyle='--',zorder=1,label='syst. error')
plt.plot(r,best_rho-best_drho_syst,color='C0',linestyle='--',zorder=1)
for key in results_dict_bar_sel:
    result=results_dict_bar_sel[key]
    nuc_result = phr.nucleus('Ar40_FB_test_'+key,18,40,ai=result['ai'],R=result['R'])
    plt.plot(r,nuc_result.charge_density(r),color='gray',zorder=-2) #,label=key
#plt.plot(r,nuc_Ar40_ref.charge_density(r),color='C1',zorder=-1,label=nuc_Ar40_ref.name)
plt.xlim(0,8.5)
plt.ylim(0,0.11)
plt.legend()
plt.title('$^{40}$Ar')
plt.xlabel('$r$ in fm')
plt.ylabel(r'$\rho$ in fm$^{-3}$')
#plt.yscale('log')

In [ ]:
print_radius(best_results_dict_bar,+1)